In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss

In [2]:
X_train = pd.read_csv('features_tr.csv',encoding='cp949')
X_test = pd.read_csv('features_te.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv', encoding='cp949')

In [3]:
X_train

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,...,occyp_type_v291,occyp_type_v292,occyp_type_v293,occyp_type_v294,occyp_type_v295,occyp_type_v296,occyp_type_v297,occyp_type_v298,occyp_type_v299,occyp_type_v300
0,0,202500.0,-13899,-4709,1,0,0,0,2.0,6.0,...,-0.089436,0.100895,-0.066787,-0.083388,0.059916,0.098018,-0.113699,-0.144115,0.017819,0.204744
1,1,247500.0,-11380,-1540,1,0,0,1,3.0,5.0,...,0.043693,0.131514,-0.007164,-0.128239,-0.021836,0.220421,-0.040256,0.082402,0.010280,0.213867
2,0,450000.0,-19087,-4434,1,0,1,0,2.0,22.0,...,-0.027730,0.081860,0.056325,0.053378,0.028166,0.134052,0.035300,-0.217725,0.002053,0.134354
3,0,202500.0,-15088,-2092,1,0,1,0,2.0,37.0,...,-0.192803,0.048254,-0.027709,-0.180056,0.022912,0.131015,0.031774,0.101893,0.036992,0.132544
4,0,157500.0,-15037,-2105,1,0,0,0,2.0,26.0,...,-0.027730,0.081860,0.056325,0.053378,0.028166,0.134052,0.035300,-0.217725,0.002053,0.134354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,2,225000.0,-12079,-1984,1,0,0,0,4.0,2.0,...,-0.190977,0.165361,0.144755,-0.179355,0.039133,0.173492,0.082324,-0.027206,0.124998,0.190396
26453,1,180000.0,-15291,-2475,1,0,0,0,2.0,47.0,...,-0.089436,0.100895,-0.066787,-0.083388,0.059916,0.098018,-0.113699,-0.144115,0.017819,0.204744
26454,0,292500.0,-10082,-2015,1,0,0,0,2.0,25.0,...,-0.190977,0.165361,0.144755,-0.179355,0.039133,0.173492,0.082324,-0.027206,0.124998,0.190396
26455,0,171000.0,-10145,-107,1,0,0,0,1.0,59.0,...,0.043693,0.131514,-0.007164,-0.128239,-0.021836,0.220421,-0.040256,0.082402,0.010280,0.213867


In [4]:
print(X_train.shape, X_test.shape, y_train.shape)

(26457, 1567) (10000, 1567) (26457, 1)


In [5]:
X_test

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,...,occyp_type_v291,occyp_type_v292,occyp_type_v293,occyp_type_v294,occyp_type_v295,occyp_type_v296,occyp_type_v297,occyp_type_v298,occyp_type_v299,occyp_type_v300
0,0,112500.0,-21990,0,1,0,1,0,2.0,60.0,...,-0.089436,0.100895,-0.066787,-0.083388,0.059916,0.098018,-0.113699,-0.144115,0.017819,0.204744
1,0,135000.0,-18964,-8671,1,0,1,0,2.0,36.0,...,-0.190977,0.165361,0.144755,-0.179355,0.039133,0.173492,0.082324,-0.027206,0.124998,0.190396
2,0,69372.0,-15887,-217,1,1,1,0,2.0,40.0,...,0.043693,0.131514,-0.007164,-0.128239,-0.021836,0.220421,-0.040256,0.082402,0.010280,0.213867
3,0,112500.0,-19270,-2531,1,1,0,0,2.0,41.0,...,-0.064925,0.053826,0.114095,-0.090608,-0.055061,0.155247,-0.065681,0.020824,-0.015149,0.152629
4,0,225000.0,-17822,-9385,1,1,0,0,2.0,8.0,...,-0.027730,0.081860,0.056325,0.053378,0.028166,0.134052,0.035300,-0.217725,0.002053,0.134354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,202500.0,-18593,-5434,1,1,1,0,2.0,19.0,...,-0.176133,-0.027530,-0.028341,0.032596,0.184138,-0.056716,0.085184,-0.063871,0.149015,0.079299
9996,0,202500.0,-10886,-1315,1,1,0,0,2.0,34.0,...,0.043693,0.131514,-0.007164,-0.128239,-0.021836,0.220421,-0.040256,0.082402,0.010280,0.213867
9997,0,292500.0,-21016,-14018,1,0,0,0,2.0,55.0,...,-0.153132,0.110237,0.125176,-0.019406,0.062362,0.097417,0.070027,-0.100196,-0.030441,0.036069
9998,0,180000.0,-16541,-1085,1,0,1,0,2.0,33.0,...,-0.089436,0.100895,-0.066787,-0.083388,0.059916,0.098018,-0.113699,-0.144115,0.017819,0.204744


# feature selection

## lgbm

In [10]:
clf = LGBMClassifier(random_state=0)

In [11]:
smf = SelectFromModel(clf, threshold='3.0*mean')
smf.fit(X_train, y_train)

C:\Users\hwangtaegyun\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hwangtaegyun\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SelectFromModel(estimator=LGBMClassifier(random_state=0), threshold='3.0*mean')

In [12]:
X_tr_new = smf.transform(X_train)
X_te_new = smf.transform(X_test)

In [13]:
features_selection_idx = smf.get_support()
features_selection_name = X_train.columns[features_selection_idx]

In [14]:
X_tr_new = pd.DataFrame(X_tr_new)
X_te_new = pd.DataFrame(X_te_new)

X_tr_new.columns = features_selection_name
X_te_new.columns = features_selection_name

In [15]:
scores = cross_val_score(clf, X_tr_new, y_train, scoring='neg_log_loss', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))
print('log_loss:', -np.mean(scores))

C:\Users\hwangtaegyun\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hwangtaegyun\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hwangtaegyun\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hwangtaegyun\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas

교차 검증별 정확도: [-0.7615 -0.7702 -0.7658 -0.7624 -0.7623]
평균 검증 정확도: -0.7644
log_loss: 0.7644468993145969


C:\Users\hwangtaegyun\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hwangtaegyun\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
X_tr_new.to_csv('sf_train.csv', index=False)
X_te_new.to_csv('sf_test.csv', index=False)

## tuning

In [6]:
import optuna
def objective_lgbm(trial):
    lgbm_random_state = 0
    lgbm_n_jobs = -1
    lgbm_num_leaves = trial.suggest_int("num_leaves", 50, 80)
    lgbm_min_child_samples = trial.suggest_int('min_child_samples',50,100)
    lgbm_max_depth = trial.suggest_int('max_depth',5,60)
    lgbm_learning_rate = trial.suggest_float('learning_rate',0.0001,0.7)
    lgbm_n_estimators = trial.suggest_int('n_estimators',50,500)
    lgbm_reg_lambda = trial.suggest_float('reg_lambda',0,1)
    lgbm_reg_alpha = trial.suggest_float('reg_alpha',0,1)
    
    
    
    
    classifier_obj = LGBMClassifier(num_leaves = lgbm_num_leaves,
                                    random_state = 0,
                                    n_jobs = lgbm_n_jobs,
                                    min_child_samples = lgbm_min_child_samples,
                                    max_depth = lgbm_max_depth,
                                    learning_rate = lgbm_learning_rate,
                                    n_estimators = lgbm_n_estimators,
                                    reg_lambda = lgbm_reg_lambda,
                                    reg_alpha = lgbm_reg_alpha,
                                    devie='gpu'
                                   )
    
    score = cross_val_score(classifier_obj, X_train,  y_train, scoring='neg_log_loss', cv=5, n_jobs=-1)
    logloss = score.mean()
    return logloss

study_lgbm = optuna.create_study(direction="maximize")
study_lgbm.optimize(objective_lgbm, n_trials=50)
print(study_lgbm.best_trial.value,study_lgbm.best_trial.params)

[I 2022-11-27 17:32:41,119] A new study created in memory with name: no-name-4b9244b9-efdf-4948-8289-5ea9ad0642f5
[I 2022-11-27 17:33:51,991] Trial 0 finished with value: -1.2505271884253926 and parameters: {'num_leaves': 56, 'min_child_samples': 95, 'max_depth': 35, 'learning_rate': 0.5259601136985205, 'n_estimators': 289, 'reg_lambda': 0.6144524198656093, 'reg_alpha': 0.3249226776884846}. Best is trial 0 with value: -1.2505271884253926.
[I 2022-11-27 17:34:13,317] Trial 1 finished with value: -1.013806457493774 and parameters: {'num_leaves': 75, 'min_child_samples': 55, 'max_depth': 5, 'learning_rate': 0.45296062113183266, 'n_estimators': 112, 'reg_lambda': 0.7469204970791506, 'reg_alpha': 0.1263265302331259}. Best is trial 1 with value: -1.013806457493774.
[I 2022-11-27 17:35:17,279] Trial 2 finished with value: -0.9569375249190083 and parameters: {'num_leaves': 71, 'min_child_samples': 85, 'max_depth': 7, 'learning_rate': 0.18706826562456674, 'n_estimators': 325, 'reg_lambda': 0.15

[I 2022-11-27 18:07:05,313] Trial 24 finished with value: -0.9139019608726258 and parameters: {'num_leaves': 60, 'min_child_samples': 79, 'max_depth': 50, 'learning_rate': 0.05338743924304145, 'n_estimators': 449, 'reg_lambda': 0.16979048169348407, 'reg_alpha': 0.6278228972072935}. Best is trial 22 with value: -0.8306149020494505.
[I 2022-11-27 18:08:17,257] Trial 25 finished with value: -1.0271361513189445 and parameters: {'num_leaves': 54, 'min_child_samples': 59, 'max_depth': 56, 'learning_rate': 0.17981612876755115, 'n_estimators': 356, 'reg_lambda': 0.5611629482509158, 'reg_alpha': 0.8752476685899889}. Best is trial 22 with value: -0.8306149020494505.
[I 2022-11-27 18:09:37,335] Trial 26 finished with value: -0.9753725735964913 and parameters: {'num_leaves': 52, 'min_child_samples': 71, 'max_depth': 41, 'learning_rate': 0.12021201381618976, 'n_estimators': 371, 'reg_lambda': 0.23719072189549556, 'reg_alpha': 0.44182086547011096}. Best is trial 22 with value: -0.8306149020494505.
[

[I 2022-11-27 18:36:46,999] Trial 48 finished with value: -1.0849815335811805 and parameters: {'num_leaves': 61, 'min_child_samples': 93, 'max_depth': 31, 'learning_rate': 0.26807230178872105, 'n_estimators': 358, 'reg_lambda': 0.09829969484143045, 'reg_alpha': 0.6837620862215356}. Best is trial 47 with value: -0.8218311886559684.
[I 2022-11-27 18:38:15,559] Trial 49 finished with value: -0.9159552374630398 and parameters: {'num_leaves': 58, 'min_child_samples': 85, 'max_depth': 22, 'learning_rate': 0.07460695968627046, 'n_estimators': 422, 'reg_lambda': 0.3257121883235028, 'reg_alpha': 0.5616603627158078}. Best is trial 47 with value: -0.8218311886559684.


-0.8218311886559684 {'num_leaves': 57, 'min_child_samples': 85, 'max_depth': 32, 'learning_rate': 0.002535699789956471, 'n_estimators': 360, 'reg_lambda': 0.3201684455423176, 'reg_alpha': 0.7028595396524745}


-0.7347345617957105 {'num_leaves': 77, 'min_child_samples': 75, 'max_depth': 24, 'learning_rate': 0.05962509116013047, 'n_estimators': 234, 'reg_lambda': 0.30710121460984924, 'reg_alpha': 0.8263393436726127}

In [6]:
lgbm_params =  {'num_leaves': 77, 'min_child_samples': 75, 'max_depth': 24, 'learning_rate': 0.05962509116013047, 'n_estimators': 234, 'reg_lambda': 0.30710121460984924, 'reg_alpha': 0.8263393436726127}

In [7]:
lgbm_parmas_nfs =  {'num_leaves': 57, 'min_child_samples': 85, 'max_depth': 32, 'learning_rate': 0.002535699789956471, 'n_estimators': 360, 'reg_lambda': 0.3201684455423176, 'reg_alpha': 0.7028595396524745}

## training

In [8]:
train_org = pd.read_csv('../open/train.csv', encoding='cp949')

In [9]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train_org, train_org['credit']):
    folds.append((train_idx, valid_idx))

In [10]:
import random

array([[ 0.0000e+00,  2.0250e+05, -1.3899e+04, ...,  0.0000e+00,
         0.0000e+00,  1.0000e+00],
       [ 1.0000e+00,  2.4750e+05, -1.1380e+04, ...,  0.0000e+00,
         0.0000e+00,  0.0000e+00],
       [ 0.0000e+00,  2.0250e+05, -1.5088e+04, ...,  0.0000e+00,
         0.0000e+00,  0.0000e+00],
       ...,
       [ 1.0000e+00,  1.8000e+05, -1.5291e+04, ...,  0.0000e+00,
         0.0000e+00,  1.0000e+00],
       [ 0.0000e+00,  2.9250e+05, -1.0082e+04, ...,  0.0000e+00,
         0.0000e+00,  0.0000e+00],
       [ 0.0000e+00,  8.1000e+04, -1.9569e+04, ...,  1.0000e+00,
         0.0000e+00,  0.0000e+00]])

In [11]:
random.seed(42)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train1, X_valid1, y_train1, y_valid1 = X_train.iloc[train_idx].values, X_train.iloc[valid_idx].values,\
                                         train_org['credit'][train_idx].values, train_org['credit'][valid_idx].values 
    lgb = LGBMClassifier(**lgbm_parmas_nfs, device = 'gpu')
    lgb.fit(X_train1, y_train1, 
            eval_set=[(X_train1, y_train1), (X_valid1, y_valid1)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')

====================================1============================================


C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's multi_logloss: 0.838808	valid_1's multi_logloss: 0.845755
[200]	training's multi_logloss: 0.811916	valid_1's multi_logloss: 0.825599
[300]	training's multi_logloss: 0.791652	valid_1's multi_logloss: 0.811996


====================================2============================================


C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's multi_logloss: 0.838496	valid_1's multi_logloss: 0.848823
[200]	training's multi_logloss: 0.810837	valid_1's multi_logloss: 0.829653
[300]	training's multi_logloss: 0.790127	valid_1's multi_logloss: 0.816648


====================================3============================================


...

[100]	training's multi_logloss: 0.839125	valid_1's multi_logloss: 0.847737
[200]	training's multi_logloss: 0.811652	valid_1's multi_logloss: 0.828
[300]	training's multi_logloss: 0.791081	valid_1's multi_logloss: 0.814474


====================================4============================================


C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's multi_logloss: 0.838752	valid_1's multi_logloss: 0.846596
[200]	training's multi_logloss: 0.810616	valid_1's multi_logloss: 0.826052
[300]	training's multi_logloss: 0.790101	valid_1's multi_logloss: 0.813082


====================================5============================================


C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\hwangtaegyun\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	training's multi_logloss: 0.839402	valid_1's multi_logloss: 0.846747
[200]	training's multi_logloss: 0.811978	valid_1's multi_logloss: 0.825811
[300]	training's multi_logloss: 0.791923	valid_1's multi_logloss: 0.812138




In [12]:
submit = pd.read_csv('../open/sample_submission.csv')

In [13]:
submit.iloc[:,1:]=0
for fold in range(5):
    submit.iloc[:,1:] += lgb_models[fold].predict_proba(X_test)/5

In [14]:
submit.to_csv('lgbm_test_submit_ensemble_w2v_nfs.csv', index=False)

In [15]:
submit.head()

,index,0,1,2
0,26457,0.104384,0.209504,0.686112
1,26458,0.117872,0.193326,0.688802
2,26459,0.106186,0.201997,0.691817
3,26460,0.112726,0.190159,0.697115
4,26461,0.126961,0.194572,0.678467
